# Accessing "State-dependent Properties"

Some experimentalists, experiment runners and theorists require access to the values
created during the cycle execution, e.g. experimentalists which require access
to the current best theory or the observed data. These data update each cycle, and
so cannot easily be set using simple `params`.

For this case, it is possible to use "state-dependent properties" in the `params`
dictionary. These are the following strings, which will be replaced during execution by
their respective current values:

- `"%observations.ivs[-1]%"`: the last observed independent variables
- `"%observations.dvs[-1]%"`: the last observed dependent variables
- `"%observations.ivs%"`: all the observed independent variables,
concatenated into a single array
- `"%observations.dvs%"`: all the observed dependent variables,
concatenated into a single array
- `"%theories[-1]%"`: the last fitted theorist
- `"%theories%"`: all the fitted theorists

In the following example, we use the `"observations.ivs"` cycle property for an
experimentalist which excludes those conditions which have
already been seen.

In [ ]:
import numpy as np
from autora.experimentalist.pipeline import make_pipeline
from autora.variable import VariableCollection, Variable
from sklearn.linear_model import LinearRegression
import pandas as pd
from functools import partial

from autora_workflow import Cycle

In [ ]:
def ground_truth(x):
    return x + 1
variables = VariableCollection(
   independent_variables=[Variable(name="x", allowed_values=range(10))],
   dependent_variables=[Variable(name="y")],
   )
random_sampler_rng = np.random.default_rng(seed=180)
def custom_random_sampler(conditions, n):
    sampled_conditions = random_sampler_rng.choice(conditions, size=n, replace=False)
    return sampled_conditions
def exclude_conditions(conditions, excluded_conditions):
    remaining_conditions = list(set(conditions) - set(excluded_conditions.flatten()))
    return remaining_conditions
unobserved_data_experimentalist = make_pipeline([
    variables.independent_variables[0].allowed_values,
    exclude_conditions,
    custom_random_sampler,
    partial(pd.DataFrame, columns=["x"])
    ]
)
example_theorist = LinearRegression()
def get_example_synthetic_experiment_runner():
    rng = np.random.default_rng(seed=180)
    def runner(x):
        return ground_truth(x) + rng.normal(0, 0.1, x.shape)
    def dataframe_runner(conditions_df: pd.DataFrame):
        observations_df = conditions_df.copy()
        observations_df["y"] = runner(conditions_df["x"])
        return observations_df
    return dataframe_runner

example_synthetic_experiment_runner = get_example_synthetic_experiment_runner()

cycle_with_state_dep_properties = Cycle(
    metadata=variables,
    theorist=example_theorist,
    experimentalist=unobserved_data_experimentalist,
    experiment_runner=example_synthetic_experiment_runner,
    params={
        "experimentalist": {
            "exclude_conditions": {"excluded_conditions": "%observations.ivs%"},
            "custom_random_sampler": {"n": 1}
        }
    }
)

Now we can run the cycler to generate conditions and run experiments. The first time round,
we have the full set of 10 possible conditions to select from, and we select "2" at random:

In [ ]:
cycle_with_state_dep_properties.run().data.conditions[-1]

We can continue to run the cycler, each time we add more to the list of "excluded" options:

In [ ]:
cycle_with_state_dep_properties.run(num_cycles=9).data.conditions[-1]

If we try to evaluate it again, the experimentalist fails, as there aren't any more
conditions which are available:

In [ ]:
cycle_with_state_dep_properties.run()  # doctest: +ELLIPSIS

In [ ]:
pd.concat(cycle_with_state_dep_properties.data.conditions, ignore_index=True)
